In [ ]:
%reload_ext autoreload
%autoreload 2

In [1]:

from torch import zeros, tensor

import os
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), "../../.."))
sys.path.append(project_root)

from utils.data_processing import get_mimic_dataset, get_mimic_dataloaders

In [2]:

# Generate the dataset and dataloaders
path = os.path.join(os.getcwd(), "data/")

train_frac = 0.475
valid_frac = 0.2
test_frac = 0.2
early_stop_frac = 0.125
batch_size = 59
flatten = False # If LR, flatten the data

dataset, train_indices, validation_indices, test_indices, early_stop_indices= get_mimic_dataset(path,
                                                                            train_frac , 
                                                                            valid_frac,
                                                                            test_frac,
                                                                            early_stop_frac,
                                                                            flatten)
                                                                            
                                                                            

In [3]:

train_loader, validation_loader, test_loader, early_stop_loader = get_mimic_dataloaders(dataset,
                                                            train_indices, 
                                                            validation_indices, 
                                                            test_indices,
                                                            early_stop_indices,
                                                            batch_size)

In [4]:
best_hyperparams ={
    'cell_size': 58,
    'hidden_size': 78, 
    'learning_rate': 0.004738759319792616,
    'num_epochs':37,
    'patience': 3,
    'batch_size': 59,
    'early_stop_frac': 0.125,
    'seed': 4410,
}
early_stop_frac = valid_frac
n_features = int(dataset.x.shape[1]/3)
X_mean = zeros(1,dataset.x.shape[2],n_features)

model_params = {k: best_hyperparams[k] for k in ['cell_size', 'hidden_size', 'batch_size']}

# Add other required parameters to model_params
model_params.update({
    'input_size': n_features,
    'X_mean': X_mean,
    'output_last': False
})



In [5]:
dataset.x.shape

torch.Size([23944, 312, 24])

In [8]:
from tqdm import tqdm
from utils.grud import *

# Initialize the model with filtered parameters
model = GRUD(**model_params)
# Train the model with Train_Model function
train_losses, test_losses ,  train_acc, test_acc = gru_trained_model_and_metadata(
            model, train_loader, early_stop_loader,
            epochs=best_hyperparams['num_epochs'], 
            patience=best_hyperparams['patience'], 
            min_delta = 1e-5,
            learning_rate=best_hyperparams['learning_rate'], 
            batch_size=best_hyperparams['batch_size'],  # This might be used in your dataloaders, not directly in Train_Model
             metadata = None)

Model Structure:  GRUD(
  (zl): Linear(in_features=286, out_features=78, bias=True)
  (rl): Linear(in_features=286, out_features=78, bias=True)
  (hl): Linear(in_features=286, out_features=78, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=78, bias=True)
  (fc): Linear(in_features=78, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model


Training Progress:   0%|          | 0/37 [00:00<?, ?it/s]

In [ ]:
import matplotlib.pyplot as plt

# Convert losses to numpy-compatible lists directly
train_losses_cpu = [float(loss) for loss in train_losses]
test_losses_cpu = [float(loss) for loss in test_losses]

# Plot the losses
plt.plot(train_losses_cpu, label='Train Loss')
plt.plot(test_losses_cpu, label='Test Loss')

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()



In [ ]:
from mimic_gru_handler import MimicInputHandlerGRU

from leakpro import LeakPro

# Read the config file
config_path = "audit.yaml"

# Prepare leakpro object
leakpro = LeakPro(MimicInputHandlerGRU, config_path)

# Run the audit 
leakpro.run_audit()